In [1]:
import warnings
warnings.filterwarnings("ignore")
from keras.engine.topology import Layer
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, GRU, Multiply, Reshape
from keras.layers import RepeatVector, Dense, Activation, Lambda, Embedding
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import keras
import numpy as np
import random
import tqdm
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import pandas as pd
# from tcn import TCN
%matplotlib inline
from keras.layers import merge, Input, Dense, TimeDistributed, Lambda                                   
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, GlobalMaxPooling2D,MaxPooling2D, Flatten, Dense, BatchNormalization, Activation, Input, GlobalAveragePooling2D,AveragePooling2D, Add
from keras.optimizers import Adam, rmsprop
from keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm
MAX_SENT_LENGTH = 55
MAX_SENTS = 24
VALIDATION_SPLIT = 0.2

Using TensorFlow backend.


In [2]:
from sklearn.preprocessing import MinMaxScaler  
from sklearn.preprocessing import StandardScaler  
from pandas import concat

std = StandardScaler()

In [3]:
import pandas as pd
#matplotlib inline
#参数初始化
discfile = '../data/data_wea_time_unify.csv'

data = pd.read_csv(discfile)

In [4]:
import h5py
import os

ground = h5py.File('../data/inte_g_12_linear.h5', 'r')
g = ground['X']

ele_zs = h5py.File('../data/inte_e_zs_12_new_linear.h5', 'r')
ezs = ele_zs['X']

ele_sd = h5py.File('../data/inte_e_sd_12_new_linear.h5', 'r')
esd = ele_sd['X']

In [5]:
g_m=[g,ezs,esd]

In [6]:
g_m=np.array(g_m).reshape(6064,16,16,3)
g=np.array(g).reshape(6064,16,16,1)

In [22]:
# from attention_keras import *
from keras.layers import LeakyReLU
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(** kwargs)
    
    def build(self, input_shape):
        assert len(input_shape)==3
        # W.shape = (time_steps, time_steps)
        self.W = self.add_weight(name='att_weight', 
                                 shape=(input_shape[1], input_shape[1]),
                                 initializer='uniform',
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs, mask=None):
        x = K.permute_dimensions(inputs, (0, 2, 1))
        # x.shape = (batch_size, seq_len, time_steps)
        a = K.softmax(K.tanh(K.dot(x, self.W)))
        a = K.permute_dimensions(a, (0, 2, 1))
#         outputs = a * inputs
# #         outputs = K.sum(outputs, axis=1)
# #         outputs = K.permute_dimensions(a * inputs, (0, 2, 1))
#         outputs = K.sum(outputs, axis=0)
    
        outputs=Multiply()([a,inputs])
        return outputs

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[2]

def create_model(learn_rate=0.001, neurons=24, traffic_n=3):    
#     交通指数数据模块
    inp=Input(shape=(16, 16, 1))
    out=Conv2D(filters=1, kernel_size=3, strides=1)(inp)
    out=MaxPooling2D(pool_size=(3, 3), strides=2)(out)
    out=Flatten()(out)

    out=Dense(3)(out)

    
#     特征组件
    sentence_input = Input(shape=(54,))
    s_input=keras.layers.concatenate([sentence_input,out ])
    embed = Reshape((54+3,1))(s_input)
#     embed = AttentionLayer()(embed)
#     embed = Reshape((54+3,1))(embed)
    l_lstm=MAttention(8,16)([embed,embed,embed])
    l_dense = TimeDistributed(Dense(32))(l_lstm)

    l_att = AttentionLayer()(l_dense)

    preds = Dense(1)(l_att)
    model_1 = Model([inp,sentence_input], preds)
    model_1.compile(optimizer=Adam(lr=learn_rate, beta_1=0.99, beta_2=0.999, decay=0.0006),
                        loss='mae')
    return model_1

model=create_model()
history=model.fit([g[:6063],data.iloc[:6063,4:58]],data['PM25_mean'][1:], epochs=20, batch_size=68,verbose=2,validation_split=0.2,shuffle=True)
# 绘制训练 & 验证的损失值
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

Train on 4850 samples, validate on 1213 samples
Epoch 1/20


InvalidArgumentError: Incompatible shapes: [3876,1] vs. [68,1]
	 [[Node: training_3/Adam/gradients/loss_3/dense_37_loss/sub_grad/BroadcastGradientArgs = BroadcastGradientArgs[T=DT_INT32, _class=["loc:@training_3/Adam/gradients/loss_3/dense_37_loss/sub_grad/Reshape"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](training_3/Adam/gradients/loss_3/dense_37_loss/sub_grad/Shape, training_3/Adam/gradients/loss_3/dense_37_loss/sub_grad/Shape_1)]]

In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 16, 16, 1)    0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 14, 14, 1)    10          input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 6, 6, 1)      0           conv2d_3[0][0]                   
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 36)           0           max_pooling2d_2[0][0]            
__________________________________________________________________________________________________
input_5 (I

In [ ]:
f = K.function(tra.inputs, [tra.get_layer('dense_27').output])

In [11]:
from keras.layers import *
import keras.backend as K


def to_mask(x, mask, mode='mul'):
    """通用mask函数
    """
    if mask is None:
        return x
    else:
        for _ in range(K.ndim(x) - K.ndim(mask)):
            mask = K.expand_dims(mask, K.ndim(mask))
        if mode == 'mul':
            return x * mask
        else:
            return x - (1 - mask) * 1e10

class OurLayer(Layer):
    """定义新的Layer，增加reuse方法，允许在定义Layer时调用现成的层
    """
    def reuse(self, layer, *args, **kwargs):
        if not layer.built:
            if len(args) > 0:
                inputs = args[0]
            else:
                inputs = kwargs['inputs']
            if isinstance(inputs, list):
                input_shape = [K.int_shape(x) for x in inputs]
            else:
                input_shape = K.int_shape(inputs)
            layer.build(input_shape)
        outputs = layer.call(*args, **kwargs)
        for w in layer.trainable_weights:
            if w not in self._trainable_weights:
                self._trainable_weights.append(w)
        for w in layer.non_trainable_weights:
            if w not in self._non_trainable_weights:
                self._non_trainable_weights.append(w)
        return outputs
class MAttention(OurLayer):
    """多头注意力机制
    """
    def __init__(self, heads=8, size_per_head=16, key_size=None,
                 mask_right=False, **kwargs):
        super(MAttention, self).__init__(**kwargs)
        self.heads = heads
        self.size_per_head = size_per_head
        self.out_dim = heads * size_per_head
        self.key_size = key_size if key_size else size_per_head
        self.mask_right = mask_right
    def build(self, input_shape):
        super(MAttention, self).build(input_shape)
        self.q_dense = Dense(self.key_size * self.heads, use_bias=False)
        self.k_dense = Dense(self.key_size * self.heads, use_bias=False)
        self.v_dense = Dense(self.out_dim, use_bias=False)
    def get_config(self):
        config = {'heads': self.heads,'size_per_head': self.size_per_head}
        base_config = super(MAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def call(self, inputs):
        q, k, v = inputs[: 3]
        v_mask, q_mask = None, None
        if len(inputs) > 3:
            v_mask = inputs[3]
            if len(inputs) > 4:
                q_mask = inputs[4]
        # 线性变换
        qw = self.reuse(self.q_dense, q)
        kw = self.reuse(self.k_dense, k)
        vw = self.reuse(self.v_dense, v)
        # 形状变换
        qw = K.reshape(qw, (-1, K.shape(qw)[1], self.heads, self.key_size))
        kw = K.reshape(kw, (-1, K.shape(kw)[1], self.heads, self.key_size))
        vw = K.reshape(vw, (-1, K.shape(vw)[1], self.heads, self.size_per_head))
        # 维度置换
        qw = K.permute_dimensions(qw, (0, 2, 1, 3))
        kw = K.permute_dimensions(kw, (0, 2, 1, 3))
        vw = K.permute_dimensions(vw, (0, 2, 1, 3))
        # Attention
        a = K.batch_dot(qw, kw, [3, 3]) / self.key_size**0.5
        a = K.permute_dimensions(a, (0, 3, 2, 1))
        a = to_mask(a, v_mask, 'add')
        a = K.permute_dimensions(a, (0, 3, 2, 1))
        if (self.mask_right is not False) or (self.mask_right is not None):
            if self.mask_right is True:
                ones = K.ones_like(a[: 1, : 1])
                mask = (ones - K.tf.matrix_band_part(ones, -1, 0)) * 1e10
                a = a - mask
            else:
                # 这种情况下，mask_right是外部传入的0/1矩阵，shape=[q_len, k_len]
                mask = (1 - K.constant(self.mask_right)) * 1e10
                mask = K.expand_dims(K.expand_dims(mask, 0), 0)
                self.mask = mask
                a = a - mask
        a = K.softmax(a)
        self.a = a
        # 完成输出
        o = K.batch_dot(a, vw, [3, 2])
        o = K.permute_dimensions(o, (0, 2, 1, 3))
        o = K.reshape(o, (-1, K.shape(o)[1], self.out_dim))
        o = to_mask(o, q_mask, 'mul')
        return o
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.out_dim)
